In [149]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from random import random
import time
import scipy as sp
import scipy.sparse.linalg
from typing import List, Tuple
from google.colab.patches import cv2_imshow
import random

In [110]:
def read_image(path:str ,  mask=False ):
  if  mask:
    #read in 2D format and convert it to binary masking format
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    _, binary_mask = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    return np.where(binary_mask == 255, 1, 0)

  else :
    #read the image
    img=cv2.imread(path)/255.0 # no need to input size , this reads input as bgr format(for both source and target image).
    return img

In [199]:
def neighbours(i: int, j: int,hs: int, ws: int,it: int, jt: int ,ht: int , wt: int , offset=False)->List[Tuple[int,int]] :
    """
    Returns 4-connected neighbours for given pixel point.
    :param i: i-th index position
    :param j: j-th index position
    :param max_i: max possible i-th index position
    :param max_j: max possible j-th index position
    """
    pairs = []
    pairt=[]

    for n in [-1, 1]:
        if 0 <= i+n <= hs-1 and 0<=it+n <=ht-1:
          pairs.append((i+n, j))
          pairt.append((it+n,jt))
        if 0 <= j+n <= ws-1 and 0<=jt+n<=wt-1:
          pairs.append((i, j+n))
          pairt.append((it,jt+n))

    return pairs,pairt

In [113]:
def mask_bounds(mask)->list[int,int,int,int]:
  h,w=mask.shape
  up,down,left,right=-1,-1,-1,-1

  for i in range(h):
    arr=mask[i,:]

    if np.sum(arr)>0 :
      # there was a mask pixel
      if up==-1:
        up=i
      else :
        down=i

  for i in range(w):
    arr=mask[:,i]

    if np.sum(arr)>0:
      #there was a mask pixel
      if left==-1:
        left=i
      else :
        right=i

  return [left,right,up,down]


# can we reduce the size function  ?

In [203]:
def startpoint_noffset(mask_limits,target_img) -> List[int,int]:
  #mask limits is a list of integers

  h,w= target_img.shape[0:2]

  if h-(mask_limits[3]-mask_limits[2]) <0 or w -(mask_limits[1]-mask_limits[0]) <0 :
       return 0  # mask size doesnot fit into , target_image,
  elif h-(mask_limits[3]-mask_limits[2])==0  and w-(mask_limits[1]-mask_limits[0])==0 :
    return 1 # mask covers whole target image, you won't have any boundary conditions from your target image.
  else:
   y,x=0,0
   y= h-(mask_limits[3]-mask_limits[2]+1)
   x= w-(mask_limits[1]-mask_limits[0]+1)
   return 2,[y,x]
  #   mask fits into the target image, and ther will be some boundary conditions available too


In [ ]:
def poisson_maskoffset(mask, image_src, image_t , mix_grad=False, cut_paste=False) ->np.ndarray :
  # mask position is shifted in translation way only.

  np.random.seed(10)
  assert mask.shape[0:2]==image_src.shape[0:2] , "mask and source images should be of the same shape"

  N=np.sum(mask) # no of masked pixels
  h_s,w_s=image_src.shape[0:2]

  r,c=np.where(mask==1 ) # masked pixel positions (r,c) , for source image


  arr_t=-np.ones(image_t.shape[0:2],dtype='int32')
  h_t,w_t=image_t.shape[0:2]
  mask_t=np.zeros((h_t,w_t),dtype=np.uint8)

  arr_s=-np.ones(image_src.shape[0:2],dtype='int32')

  arr_s[mask>0]=np.arange(N) # mapping indices to all the masked pixels in the source image

  # first get the bounds for mask area:

  limit=mask_bounds(mask) #  [left,right,up,down]

  status,available_offset=startpoint_noffset(limit,mask)


  if status==0:
    raise Exception("Blend cannot happen , as mask shape is bigger than target image ")
  else :
    x=np.random.choice(available_offset[1]+1,1).astype(int) # choosing one value out of all possible horizontal movement , manual  choice [0]
    y=np.random.choice(available_offset[0]+1,1).astype(int) # choosing one value out of all possible vertical movement , manual choice [0]
    # random choice can be changedd based on the value fed fo np.random.see(?)

    # translating mask from source object to , target image .
    mask_t[y[0]:y[0]+(limit[3]-limit[2]+1),x[0]:x[0]+(limit[1]-limit[0]+1) ] =mask [limit[2]:limit[3]+1 ,limit[0]:limit[1]+1]
    #mask generation is done
    arr_t[mask_t>0]=arr_s[mask>0]

  #before poisson blend , let's try copy pasting .

  print(N)

  if cut_paste:
    cut_copy=image_t.copy()
    for i in range(N):
      #print(i)
      R,C=r[i],c[i]
      index=arr_s[R][C] # index value of the pixel from source image
      one,two,three=image_src[R][C][0],image_src[R][C][1], image_src[R][C][2]

      Rt,Ct=np.where(arr_t==index)
      Rt=Rt[0]
      Ct=Ct[0]
      #print(Rt,Ct)
      cut_copy[Rt][Ct][0],cut_copy[Rt][Ct][1],cut_copy[Rt][Ct][2]=one,two,three
    return   cut_copy


  #time to apply poisson blend ,
  else:
    poisson_blend=image_t.copy()
    b = np.zeros(int(4*N))
    A=sp.sparse.lil_matrix((int(4*N),N))
    ind=0
    for i in range(N):

      #print(i)

      R,C=r[i],c[i] # we got the pixel locations from source image

      index=arr_s[R][C] # index value of the pixel from source image

      Rt,Ct=np.where(arr_t==index)
      Rt=Rt[0]
      Ct=Ct[0]
      #time to find their neghbour  :
      pairs,pairt=neighbours(R,C,h_s,w_s,Rt,Ct,h_t,w_t,False)
      # R,c or Rt,Ct are mapped to same index
      for j in range(len(pairs)) :
        #i will get tuple
        A[ind,arr_t[Rt][Ct]]=1

        nr_s,nc_s=pairs[j]
        nr_t,nc_t=pairt[j]

        grads=image_src[R][C] - image_src[nr_s][nc_s]
        gradt=image_t[Rt][Ct]-image_t[nr_t][nc_t]
        #print(grad)
        if mix_grad:
          b[ind]=grads if abs(grads) > abs(gradt) else gradt
        else :
          b[ind]=grads
        if arr_t[nr_t][nc_t]==-1:
          # it is a specific boundary value , it will not change whether we are trying mix grad or normal grad fusion .
          b[ind]+=(image_t[nr_t][nc_t]/1)
        else :
          # it is a unknown pixel value which needs to be determined
          A[ind,arr_t[nr_t][nc_t]]=-1

        ind+=1
    A = sp.sparse.csr_matrix(A)
    X = sp.sparse.linalg.lsqr(A, b)[0]

    New_target=image_t.copy()

    for i in range(N):
      R,C=r[i],c[i]
      index=arr_s[R][C] # we got the index , find their location , in the target image
      new_r,new_c=np.where(arr_t==index)
      New_target[new_r[0]][new_c[0]]=X[index]

    return np.clip(New_target,0,1)


In [187]:
mask=read_image('/content/mask1.jpg',True)
bg = read_image('/content/img1.jpg')
obj = read_image('/content/img2.jpg')

In [188]:
g=poisson_maskoffset(mask, obj, bg , False,True)

18347


In [ ]:
cv2_imshow(np.uint8(g*255.0)) # cut paste image

In [ ]:
take=np.zeros(bg.shape)
for i in range(3):
  take[:,:,i]=poisson_maskoffset(mask,obj[:,:,i],bg[:,:,i].copy(),True,False)
  print(i)

In [ ]:
cv2_imshow(np.uint8(take*255.0))
#mix gradient blending

In [ ]:
take=np.zeros(bg.shape)
for i in range(3):
  take[:,:,i]=poisson_maskoffset(mask,obj[:,:,i],bg[:,:,i].copy(),False,False)
  print(i)

In [ ]:
cv2_imshow(np.uint8(take*255.0))
#poisson blending

In [195]:
"""# next thing how to map indices to them , so that we can apply filtering in better way .
def  poison_blend(mask,image_src,image_t,mix_grad=False ,offset=[0,0]):
  #mask and source  image should be of same size

  assert mask.shape==image_src.shape , "mask and the source images should  be of the same shape"
  #
  #limit=mask_bounds(mask)

  #start_noffset=startpoint_noffset(limit,image_t,image_src)

  N=np.sum(mask) # no of masked pixels


  r,c=np.where(mask==1) # pixels positions (r,c) , where it is masked

  max_R,max_C=image_src.shape # mask and source image are of same sizefdc bcdvgs

  arr=-np.ones(image_t.shape[0:2] , dtype='int32')

  #change  the mask offset as per the target image
  #mask_t=target_mask(mask,image_t,limit)

  arr[mask>0]=np.arange(N) # mapping indices to all masked pixels , it will help us in creating arr

  b = np.zeros(int(4*N))
  A=sp.sparse.lil_matrix((int(4*N),N))

  ind=0

  for i in range(N):
    R,C=r[i],c[i] # got the pixels position

    #print(R,C)
    #print(arr[R][C])
    index=arr[R][C] # index value of the pixel

    #get its neighbors
    pair=neighbours(R,C,max_R-1,max_C-1)

    #A[ind,arr[R][C]]=1

    for j in pair :
      #i will get tuple
      A[ind,arr[R][C]]=1
      nr,nc=j
      grads=image_src[R][C] - image_src[nr][nc]
      gradt=image_t[R][C]-image_t[nr][nc]
      #print(grad)
      if mix_grad:
        b[ind]=grads if abs(grads) > abs(gradt) else gradt
      else :
        b[ind]=grads
      if arr[nr][nc]==-1:
        # it is a specific boundary value , it will not change whether we are trying mix grad or normal grad fusion .
        b[ind]+=(image_t[nr][nc]/1)
      else :
        # it is a unknown pixel value which needs to be determined
        A[ind,arr[nr][nc]]=-1

      ind+=1


  A = sp.sparse.csr_matrix(A)
  X = sp.sparse.linalg.lsqr(A, b)[0]

  New_target=image_t.copy()

  for i in range(N):
    R,C=r[i],c[i]
    New_target[R][C]=X[arr[R][C]]

  return np.clip(New_target, 0, 1)
"""


'# next thing how to map indices to them , so that we can apply filtering in better way .\ndef  poison_blend(mask,image_src,image_t,mix_grad=False ,offset=[0,0]):\n  #mask and source  image should be of same size\n\n  assert mask.shape==image_src.shape , "mask and the source images should  be of the same shape"\n  #\n  #limit=mask_bounds(mask)\n\n  #start_noffset=startpoint_noffset(limit,image_t,image_src)\n\n  N=np.sum(mask) # no of masked pixels\n\n\n  r,c=np.where(mask==1) # pixels positions (r,c) , where it is masked\n\n  max_R,max_C=image_src.shape # mask and source image are of same sizefdc bcdvgs\n\n  arr=-np.ones(image_t.shape[0:2] , dtype=\'int32\')\n  \n  #change  the mask offset as per the target image \n  #mask_t=target_mask(mask,image_t,limit)\n\n  arr[mask>0]=np.arange(N) # mapping indices to all masked pixels , it will help us in creating arr\n\n  b = np.zeros(int(4*N))\n  A=sp.sparse.lil_matrix((int(4*N),N))\n\n  ind=0\n\n  for i in range(N):\n    R,C=r[i],c[i] # got th